# MOVIE-RECOMMENDER: Set Up database with SQLite3

This is the pipeline description of the database schema used for our web app.  
As a first step, we will be running the server locally. For that reason, sqlite3 allows us to get a fast start to begin testing the system, and in the future we can adapt our database to an online platform.

## Necessary imports

In [ ]:
# Imports
import os
import sqlite3
import requests
from bs4 import BeautifulSoup
from ast import literal_eval
import numpy as np

## Creating the DataBase

This code corresponds to the 'create_db.py' script.  
Necessary variables from the 'config.py' file are also included for simplicity

In [ ]:
# Set our configuration parameters
current_dir = os.getcwd()
DB_FILE = f'{current_dir}/movies_db.db'

MOVIES_PER_ROW = 5
DATA_PATH = f"{current_dir}/data/"
MOVIE_DB_URL = "https://www.themoviedb.org/"
NO_IMAGE = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Image-missing.svg/480px-Image-missing.svg.png"

In [ ]:
# OPEN // CREATE database
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

In [ ]:
# Table for MOVIES
cursor.execute("""
	CREATE TABLE IF NOT EXISTS movies(
		id INTEGER PRIMARY KEY,
		movieDB_id INTEGER NOT NULL UNIQUE,
		title TEXT NOT NULL UNIQUE,
		duration NOT NULL,
		vote_counts INTEGER NOT NULL,
		vote_average NOT NULL,
		release_date NOT NULL,
		poster_path,
		popularity NOT NULL,
		director NOT NULL
		)
	""")

In [ ]:
# Table for ACTORS
cursor.execute("""
	CREATE TABLE IF NOT EXISTS actors(
		id INTEGER PRIMARY KEY,
		name NOT NULL UNIQUE
		)
	""")

In [ ]:
# Table for GENRES
cursor.execute("""
	CREATE TABLE IF NOT EXISTS genres(
		id INTEGER PRIMARY KEY,
		name NOT NULL UNIQUE
		)
	""")

In [ ]:
# Table for KEYWORDS
cursor.execute("""
	CREATE TABLE IF NOT EXISTS keywords(
		id INTEGER PRIMARY KEY,
		name NOT NULL UNIQUE
		)
	""")

In [ ]:
# Relational table MOVIE-GENRE
cursor.execute("""
	CREATE TABLE IF NOT EXISTS movie_genres(
		id INTEGER PRIMARY KEY,
		movie_id INTEGER NOT NULL,
		genre_id INTEGER NOT NULL,
		FOREIGN KEY (movie_id) REFERENCES movies (id)
		FOREIGN KEY (genre_id) REFERENCES genres (id)
		)
	""")


In [ ]:
# Relational table MOVIE-ACTOR
cursor.execute("""
	CREATE TABLE IF NOT EXISTS movie_actors(
		id INTEGER PRIMARY KEY,
		movie_id INTEGER NOT NULL,
		actor_id INTEGER NOT NULL,
		FOREIGN KEY (movie_id) REFERENCES movies (id)
		FOREIGN KEY (actor_id) REFERENCES actors (id)
		)
	""")

In [ ]:
# Relational table MOVIE-KEYWORD
cursor.execute("""
	CREATE TABLE IF NOT EXISTS movie_keywords(
		id INTEGER PRIMARY KEY,
		movie_id INTEGER NOT NULL,
		keyword_id INTEGER NOT NULL,
		FOREIGN KEY (movie_id) REFERENCES movies (id)
		FOREIGN KEY (keyword_id) REFERENCES keywords (id)
		)
	""")

In [ ]:
# Commit changes to the database
connection.commit()

## Populating the DataBase


We are going to populate the database with data from movies downloaded from a Kaggle Dataset:   

In the future we will replace this with a web scraper to include new titles to our dataset.

In [5]:
def get_poster_path(movie_id):
	r = requests.get(f"{c.MOVIE_DB_URL}/movie/{movie_id}")
	soup = BeautifulSoup(r.content, features = 'html.parser')
	for item in soup.findAll('meta'):
		if str(item).split()[1].endswith('.jpg"'):
			return (str(item).split()[1].strip('content=').strip('"'))


In [ ]:
def get_director(movie_id, credits):
	crew = credits.loc[credits['id'] == int(movie_id)]['crew']
	crew = literal_eval(crew.values[0])
	for c in crew:
		if c['job'] == 'Director':
			return c['name']
	return "No Name"

In [ ]:
def get_main_actors(movie_id, credits):
	cast = credits.loc[credits['id'] == int(movie_id)]['cast']
	cast = literal_eval(cast.values[0])
	actors = [c['name'] for c in cast]
	if len(actors) > 5:
		actors = actors[0:5]
	return actors

In [ ]:
def get_genres(movie_id, movies):
	genres = movies.loc[movies['id'] == str(movie_id)]['genres'].values[0]
	genres = literal_eval(genres)
	genre_list = [genre['name'] for genre in genres]
	return genre_list

In [ ]:
def get_keywords(movie_id, keywords_df):
	keywords = keywords_df.loc[keywords_df['id'] == int(movie_id)]['keywords']
	keywords = literal_eval(keywords.values[0])
	keyword_list = [k['name'] for k in keywords]
	return keyword_list

In [ ]:
connection = sqlite3.connect(c.DB_FILE)
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

In [ ]:
# Read movies from CSV file
movies = pd.read_csv(c.DATA_PATH + 'movies_metadata.csv', low_memory = False)
# 10% most voted movies
most_voted = movies['vote_count'].quantile(0.9)
most_voted_movies = movies.loc[movies['vote_count'] >= most_voted]

# Read extra data (crew, cast and keywords)
credits = pd.read_csv(c.DATA_PATH + 'credits.csv', low_memory = False)
keywords_df = pd.read_csv(c.DATA_PATH + 'keywords.csv')

In [ ]:
for idx , movie in most_voted_movies.iterrows():

		movieDB_id = movie['id']
		title = movie['title']
		duration = movie['runtime']
		vote_counts = movie['vote_count']
		vote_average = movie['vote_average']
		release_date = movie['release_date']
		popularity = movie['popularity']
		poster_path = h.get_poster_path(movieDB_id)
		director = h.get_director(movieDB_id, credits)
		actors = h.get_main_actors(movieDB_id, credits)
		genres = h.get_genres(movieDB_id, most_voted_movies)
		keywords = h.get_keywords(movieDB_id, keywords_df)

		print(f"{idx} {title}")
		try:
			# print(f"Adding movie: {title} with ID:{movieDB_id}")
			cursor.execute("""
				INSERT INTO movies (movieDB_id, title, duration, vote_counts, vote_average, 
				release_date, poster_path, popularity, director) VALUES (?,?,?,?,?,?,?,?,?)
				""", (movieDB_id, title, duration, vote_counts, vote_average, release_date, poster_path, popularity, director))

		except Exception as e:
			# print(f"This movie:{title} already existed")
			# print(e) 
			pass

		for actor in actors:	
			try:
				# print(f"Adding actor: {actor}")
				cursor.execute("""
					INSERT INTO actors (name) VALUES (?)
					""", (actor,))
			except Exception as e:
				# print(f"This actor: {actor} already existed")
				# print(e) 	
				pass

		for genre in genres:	
			try:
				# print(f"Adding genre: {genre}")
				cursor.execute("""
					INSERT INTO genres (name) VALUES (?)
					""", (genre,))
			except Exception as e:
				# print(f"This genre: {genre} already existed")
				# print(e) 	
				pass


		for keyword in keywords:	
			try:
				# print(f"Adding genre: {genre}")
				cursor.execute("""
					INSERT INTO keywords (name) VALUES (?)
					""", (keyword,))
			except Exception as e:
				# print(f"This genre: {genre} already existed")
				# print(e) 	
				pass


		cursor.execute("""SELECT id FROM movies WHERE title == ?""", (title,))
		movie_id = cursor.fetchone()[0]
		for genre in genres:
			cursor.execute("""SELECT id FROM genres WHERE name == ?""", (genre,))
			genre_id = cursor.fetchone()[0]

			cursor.execute("""
					INSERT INTO movie_genres (movie_id, genre_id) VALUES (?,?)
					""", (movie_id,genre_id))

		for actor in actors:
			cursor.execute("""SELECT id FROM actors WHERE name == ?""", (actor,))
			actor_id = cursor.fetchone()[0]
			cursor.execute("""
					INSERT INTO movie_actors (movie_id, actor_id) VALUES (?,?)
					""", (movie_id,actor_id))

		for keyword in keywords:
			cursor.execute("""SELECT id FROM keywords WHERE name == ?""", (keyword,))
			keyword_id = cursor.fetchone()[0]
			cursor.execute("""
					INSERT INTO movie_keywords (movie_id, keyword_id) VALUES (?,?)
					""", (movie_id,keyword_id))	

connection.commit()

## Add extra fields and values

In [ ]:
# Establish connection with our database
connection = sqlite3.connect(c.DB_FILE)
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

In [ ]:
def weighted_rating(vote_counts, vote_average):
	""" COMPUTE WEIGHTED AVERAGE USING THE FORMULA:
		(v/(v+m) * R) + (m/(v+m) * C)
		v is the number of votes for the movie
		m is the minimum votes required to be listed in the chart
		R is the average rating of the movie
		C is the mean vote across the whole report
	"""
	v = vote_counts
	m = np.min(v)
	R = vote_average
	C = np.mean(R)
	f = 5 # Punishment for low votes

	return ((v/(v+f*m) * R)
	     +  (f*m/(v+f*m) * C))

In [ ]:
# Recompute the Score based on a weighted metric
cursor.execute("""SELECT id, vote_counts, vote_average FROM movies""")
movies = cursor.fetchall()

ids = [movie['id'] for movie in movies]
vote_counts = [int(float(movie['vote_counts'])) for movie in movies]
vote_average = [float(movie['vote_average']) for movie in movies]

scores = weighted_rating(vote_counts, vote_average)

In [ ]:
cursor.execute("""ALTER TABLE movies ADD COLUMN scores""")

In [ ]:
for idx, score in zip(ids,scores):
	cursor.execute("""UPDATE movies SET scores = ? WHERE id = ?""", (round(score,2),idx))

connection.commit()